In [1]:
username = "Alex"
pathtorepo = "C:\\Users\\" *username *  "\\Desktop\\"
using Pkg
Pkg.activate(pathtorepo * "dynamical-systems\\env\\integrate\\")
include(pathtorepo * "dynamical-systems\\system.jl")


using StaticArrays, DifferentialEquations, DynamicalSystems, JLD
E, x, y  = -40..40, -5..5,  -5..5
box = E × x × y
using GLMakie
using LinearAlgebra, LinearSolve,Statistics

cd("C:\\Users\\" *username *  "\\Desktop\\dynamical-systems\\Tsodyks Markram\\Levanova\\3 набор параметров\\Сопоставление с матконт\\файлы matlab")
I0_hom = load("I0_hom_hom.jld")["data"]
u0_hom = load("U0_hom_hom.jld")["data"]
I0_hom = I0_hom[:]
u0_hom = u0_hom[:];

  Activating project at `C:\Users\Alex\Desktop\dynamical-systems\env\integrate`

In [2]:
function condition(u, t, integrator)
    x = Vector(u)
    b = x - fp[1]
    b = Vector(b)
    linprob = LinearProblem(A, b)
    linsolve = solve(linprob)
    return norm(linsolve.u, Inf) - ϵ_box
end
affect!(integrator) = terminate!(integrator)
function get_norm_αs(u)
    x = Vector(u)
    b = x - fp[1]
    b = Vector(b)
    linprob = LinearProblem(A, b)
    linsolve = solve(linprob)
    return norm(linsolve.u, Inf), linsolve
end
function color_alpha(alpha)
    if alpha > 0.0
        return "red"
    else
        return "blue"
    end
end;

In [3]:
const τ = 0.013;  const τD = 0.07993;  const τy = 3.3;  const J = 3.07;  const β = 0.300
const xthr = 0.75; const ythr = 0.4; const α = 1.58; const ΔU0 = 0.305

t = 2000.0; tspan = [0.0, t];

In [25]:
# index point from homoclinic curve
index_hom = 1
I0 = I0_hom[index_hom]
U0 = u0_hom[index_hom]

p = SA[α, τ, τD, τy, J, xthr, ythr, U0, ΔU0, β, I0]
u0 = [0.0, 0.0, 0.0]

ds = CoupledODEs(TM, u0, p)
fp, _, _ = fixedpoints(ds, box, jacob_TM_)

ϵ_box = 1e-4
Ju0 = jacob_TM_(fp[1], p, 0);
fp_ = fp[1]

eigen_val_vec = eigen(Ju0)
eigen_vectors = eigen_val_vec.vectors

v1 = real(eigen_vectors[:, 1])
v2 = real(eigen_vectors[:, 2])
v3 = imag(eigen_vectors[:, 3])

v1 = normalize(v1, 2)
v1t = transpose(v1)

v2 = normalize(v2, 2)
v2t = transpose(v2)

v3 = normalize(v3, 2)
v3t = transpose(v3)

A = transpose([v1t; v2t; v3t])
A = Matrix(A)

3×3 Matrix{Float64}:
  0.998728    0.999899     0.0
 -0.0489879  -0.0141814    0.984697
  0.0119121   0.000705852  0.174274

In [26]:
total_number_points = 50
number_points_on_side = Int((total_number_points) / 1)

dots_right_side = zeros(number_points_on_side, length(u0))
array_α_vec_right = zeros(number_points_on_side, length(u0))

dots_left_side = zeros(number_points_on_side, length(u0))
array_α_vec_left = zeros(number_points_on_side, length(u0))

dots_up_side = zeros(number_points_on_side, length(u0))
array_α_vec_up = zeros(number_points_on_side, length(u0))

dots_down_side = zeros(number_points_on_side, length(u0))
array_α_vec_down = zeros(number_points_on_side, length(u0));

In [27]:
# case α1 = 0; α2 = ϵ_box; α3 = [-ϵ_box; ϵ_box]
n = 1
N = total_number_points
while n <= number_points_on_side

    α1 = 0.0
    α2 = ϵ_box
    α3 = - ϵ_box + 2 * ϵ_box * n / N
    α_vec = [α1, α2, α3]
    point = fp_ + A * α_vec
    
    dots_right_side[n, :] = point
    _, α_vec = get_norm_αs( dots_right_side[n, :] )
    array_α_vec_right[n, :] = α_vec
    n+=1
end

# case α1 = 0; α2 = -ϵ_box; α3 = [-ϵ_box; ϵ_box]
n = 1
N = total_number_points
while n <= number_points_on_side

    α1 = 0.0
    α2 = -ϵ_box
    α3 = - ϵ_box + 2 * ϵ_box * n / N
    α_vec = [α1, α2, α3]
    point = fp_ + A * α_vec
    
    dots_left_side[n, :] = point
    _, α_vec = get_norm_αs( dots_left_side[n, :] )
    array_α_vec_left[n, :] = α_vec
    n+=1
end

# case α1 = 0; α2 = [-ϵ_box; ϵ_box]; α3 = ϵ_box
n = 1
N = total_number_points
while n <= number_points_on_side

    α1 = 0.0
    α2 = - ϵ_box + 2 * ϵ_box * n / N
    α3 = ϵ_box
    α_vec = [α1, α2, α3]
    point = fp_ + A * α_vec
    
    dots_up_side[n, :] = point
    _, α_vec = get_norm_αs( dots_up_side[n, :] )
    array_α_vec_up[n, :] = α_vec
    n+=1
end

# case α1 = 0; α2 = [-ϵ_box; ϵ_box]; α3 = -ϵ_box
n = 1
N = total_number_points
while n <= number_points_on_side

    α1 = 0.0
    α2 = - ϵ_box + 2 * ϵ_box * n / N
    α3 = -ϵ_box
    α_vec = [α1, α2, α3]
    point = fp_ + A * α_vec
    
    dots_down_side[n, :] = point
    _, α_vec = get_norm_αs( dots_down_side[n, :] )
    array_α_vec_down[n, :] = α_vec
    n+=1
end;

In [29]:
Es = Float64[]; xs = Float64[]; ys = Float64[]
α1s =  Float64[]; α2s =  Float64[]; α3s =  Float64[]

colors = []
check_events = []
time_events = Float64[]

Float64[]

In [30]:
cb = ContinuousCallback(condition, nothing, affect!)

index = 7
u0 = SA[dots_up_side[index, 1], dots_up_side[index, 2], dots_up_side[index, 3]]
prob = ODEProblem(TM, u0, tspan, p)
sol = solve(prob, alg = Vern9(), abstol = 1e-14, reltol = 1e-14, callback = cb)

retcode: Terminated
Interpolation: specialized 9th order lazy interpolation
t: 34266-element Vector{Float64}:
   0.0
   0.023861421042473675
   0.03849888440089076
   0.05554109608485222
   0.07154504729931804
   0.08836575522347312
   0.10491647288217588
   0.12263216463151544
   0.1387249292196879
   0.156875588816766
   ⋮
 358.40614518970483
 358.42797939449906
 358.4473507280917
 358.46824222702446
 358.48750751578496
 358.5090251529936
 358.5321170601253
 358.54074921968635
 358.54074921968635
u: 34266-element Vector{SVector{3, Float64}}:
 [8.345736458879138, 0.7385941554312518, 0.4383159369490734]
 [8.349329253467802, 0.7385430940391015, 0.4383184539282618]
 [8.35155161126201, 0.7385058988487473, 0.43831901709897997]
 [8.354004927209436, 0.738459022226449, 0.43831860998017336]
 [8.356046217169066, 0.7384135825324631, 0.4383171338010143]
 [8.357778958823035, 0.738366855101744, 0.43831444020152505]
 [8.358946281025329, 0.7383245277948356, 0.4383107101176173]
 [8.359486703117968, 0.

In [31]:
norm_, alphas = get_norm_αs(sol[end])

(0.0001000000000013857, [4.974945489845076e-5, 0.0001000000000013857, -5.142596733536937e-5])

In [32]:
ϵ_box

0.0001

In [33]:
alphas[1]

4.974945489845076e-5

In [37]:
# last points in phase space
idx, idy, idz = 2, 3, 1
tstart, tend = 34266-200, 34266
f = Figure(resolution = (700, 700))

ax = LScene(f[1, 1], show_axis = true)
    
scale!(ax.scene, 50, 50, 1)

lines!(sol[idx, tstart:tend], sol[idy, tstart:tend], sol[idz, tstart:tend], markersize = 7.0, color = :deeppink)

scatter!(fp[1][idx], fp[1][idy], fp[1][idz], color = :blue, markersize = 7.0)

display(GLMakie.Screen(), f);

In [10]:
for index in range(1, length(sol), step = 1)
    norm_, linsolve = get_norm_αs(sol[index])
    push!(α1s, linsolve[1])
    push!(α2s, linsolve[2])
    push!(α3s, linsolve[3])
end